In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [21]:
# Load 1-9 digit data
with open('digits_1_to_9_train.pkl', 'rb') as f:
    train = pickle.load(f) # List of tuples (image, label)

with open('digits_1_to_9_test.pkl', 'rb') as f:
    test = pickle.load(f) # List of tuples (image, label)

# Split into images and labels
train_images, train_labels = zip(*train)
test_images, test_labels = zip(*test)

train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)


In [ ]:
# Reshape the data to be 28x28x1
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))

print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

In [23]:
# Create validation set
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [24]:
# Normalize data by pixel value
train_images, test_images = train_images / 255.0, test_images / 255.0

In [84]:
# Define the model
model = Sequential([

    Conv2D(32, (3,3), activation="relu", input_shape=(28,28,1)),

    MaxPooling2D(2,2), 

    Dropout(0.1),

    Conv2D(64, (3,3), activation="relu"),

    MaxPooling2D(2,2),

    Dropout(0.1),

    Conv2D(64, (3,3), activation="relu"),

    Flatten(),
    
    Dense(128, activation="relu"),

    Dropout(0.1),

    Dense(128, activation="relu"),

    Dense(10, activation="softmax") #10 classes, accounting for future finetuning of '0' digit class
])

# Create the model
model.compile(optimizer=Adam(learning_rate=0.0003), loss="sparse_categorical_crossentropy" , metrics=["accuracy"])

# Summary of the model
model.summary()

Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_87 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_50 (MaxPooling2D) │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_52 (Dropout)            │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_88 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_51 (MaxPooling2D) │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_53 (Dropout)            │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_89 (Conv2D)              │ (None, 3, 3, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_25 (Flatten)            │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_72 (Dense)                │ (None, 128)            │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_54 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_73 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_74 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 147,402 (575.79 KB)

 Trainable params: 147,402 (575.79 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels), callbacks=[early_stopping])

# Evaluate model accuracy on 1-9 digit test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

In [78]:
# Save model - Test accuracy on digits 1-9: 0.9924612045288086
model.save('cnn_model.keras')